In [1]:
import shutil
import requests
import sys
from langchain_core.language_models import BaseChatModel
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.document_loaders import PyPDFLoader
import os
import gradio as gr

In [8]:
# HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
HF_TOKEN = ""
os.environ['OPENAI_API_KEY'] = ""
from huggingface_hub import login
login(HF_TOKEN)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models import ChatHuggingFace

llm = HuggingFaceHub(
    repo_id= "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    huggingfacehub_api_token = HF_TOKEN,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

chat_model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_text_gen_inference.HuggingFaceTextGenInference` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_a

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
from langchain.prompts import ChatPromptTemplate

resume_summary_generation_prompt = """
Your task is to summarize in 1500 characters the experiences and skills from a given resume in a format that 
facilitates easy comparison with a job description.

Provide your summary as follows:

Output:::
Summary: (your summary of the experience from the resume, structured for easy comparison with a job description)

Now here is the resume.

Resume: {resume}\n
Output:::"""

resume_summary_generation_prompt = ChatPromptTemplate.from_template(resume_summary_generation_prompt)
resume_summary_generation_agent = resume_summary_generation_prompt | chat_model




jobDescription_summary_generation_prompt = """
Your task is to summarize in 1000 characters the provided job description in a concise format that 
can be easily compared with the summary of a resume.

Provide your job description summary as follows:

Output:::
Job Description Summary: (your summarized version of the job description, structured for easy comparison with the summary of a resume)

Now here is job description.

job description: {jobDescription}\n
Output:::"""

jobDescription_summary_generation_prompt = ChatPromptTemplate.from_template(jobDescription_summary_generation_prompt)
jobDescription_summary_generation_agent = jobDescription_summary_generation_prompt | chat_model


EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a resume to evaluate, a job description that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the resume strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
A Persons resume for a Job.

###Resume to evaluate:
{resume}

###Job Description (Score 5):
{jobDescription}

###Score Rubrics:
[does the resume contain the Skills, Qualifications, and Experiences based on the Job Description?]
Score 1: The resume contains completely no Skills, Qualifications, and Experiences.
Score 2: The resume contains mostly no Skills, Qualifications, and Experiences.
Score 3: The resume contains somwWhat Skills, Qualifications, and Experiences.
Score 4: The resume contains mostly Skills, Qualifications, and Experiences.
Score 5: The resume contains completely Skills, Qualifications, and Experiences.

###Feedback:"""


from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)


from langchain.schema import SystemMessage
evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
# from langchain_community.chat_models import ChatOpenAI
eval_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [11]:
def loadTextFromResume(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    resume = ""
    i = 0
    for page in pages:
        if i != 0:
            resume += "\n"
        resume += page.page_content
        i += 1
    return resume

In [22]:
def SummarizeResume(resume):
    output = resume_summary_generation_agent.invoke({"resume": resume}).content
    try:
        summary = output.split("Summary:")[2].split("Answer: ")[0]
        return summary
    except Exception as e:
        return e

In [23]:
def SummarizeJobDescription(jobDescription):
    output = jobDescription_summary_generation_agent.invoke({"jobDescription":jobDescription}).content
    try:
        summary = output.split("Summary:")[2].split("Answer: ")[0]
        return summary
    except Exception as e:
        # return "No summary of the resume...."
        return e

In [32]:
def evaluate(resumeSummary, jobDescriptionSummary):
    eval_prompt_rag = evaluation_prompt_template.format_messages(
        resume = resumeSummary,
        jobDescription =jobDescriptionSummary,
       )
    eval_result = eval_chat_model.invoke(eval_prompt_rag)
    try:
        resume_feedback, resume_score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        return [resume_feedback, resume_score]
    except Exception as e:
        return e

In [35]:
def predict(file, jobDescription):
    
    #Loads Text from resume
    resume = loadTextFromResume(file)

    #summarizes resume text for easy Job description comparison
    resumeSummary = SummarizeResume(resume) 

    #Summarize Job description
    jobDescriptionSummary = SummarizeJobDescription(jobDescription)

    results = evaluate(resumeSummary, jobDescriptionSummary)
    
    res = f"Match Score: {results[1]}"
    res += "\n" + f"feedback: {results[0]}"
    return res

    
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.File(label="Upload Resume pdf only"),
        gr.Textbox(label="Job Description"),
    ],
    outputs="text",
    title="JobFitScore"
)

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://9af5fc6393a1bec23b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
